### Beautifulsoup

---

Asi jeden z těch nejznámějších parserů určených k rozdělování `html`, `xml` atd. Opět musíme nejprve nainstalovat (jedná se o balíček třetí strany).

<br>

V našem virtuálním pracovním prostředí:
```bash
$ pip3 install beautifulsoup
$ pip3 freeze > requirements.txt
```

<br>

#### Vstupní hodnoty

---

In [2]:
import requests
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'bs4'

In [6]:
cislo = 1
url = f"https://heroes3.cz/hraci/index.php?page={cislo}&order=&razeni=DESC"
jmeno_souboru = f"vystup_tab_{cislo}.csv"

<br>

#### Odeslání požadavku

---

In [7]:
odp_serveru = requests.get(url)

In [8]:
type(odp_serveru.text)

str

In [9]:
print(odp_serveru.text[:25])

<!DOCTYPE html>
<html la


<br>

#### Parsování html

---

In [10]:
help(BeautifulSoup)

NameError: name 'BeautifulSoup' is not defined

In [ ]:
soup = BeautifulSoup(odp_serveru.text, 'html.parser')

In [ ]:
print(type(soup))

In [ ]:
dir(soup)

In [ ]:
print(soup.prettify())


<br>

#### Rozdělené html podle tagů

---

Nyní máme sice `html` soubor rozdělený, ale potřebujeme vybrat jen ta data, která nás zajímají.

1. pořadí
2. jméno
3. vítězství
4. celkem her

<br>

Abychom správně dohledali obsah těchto tagů, budeme je muset najít ve struktuře zdrojového kódu, v prohlížeči (režim `inspect`).

<br>

####  Hledání rodičovského tagu

---

Jakmile najdeme rodičovský tag, můžeme jej ověřit u jednotlivých údajů (`CSS path`):
```html
form > table.tab_top > tbody > tr > td
```

<br>

Protože nás zajímají všichni hráči v tabulce, použijeme celý element `table`. Jakmile jej najdeme, můžeme si dále ověřit jestli nemá nějaký atribut, který by nám jej pomohl selektovat:
```html
<table class="tab_top">
    ...
</table>
```

<br>

#### Tagy potomků

---

Postupně procházíme všechny dědičné tagy a odpovíme společně na tyto otázky: https://www.w3schools.com/TAGS/tag_table.asp
1. Co představuje tag: `<td></td>`?
2. Co představuje tag: `<tr></tr>`?
3. Co představuje tag: `<table></table>`?

<br>

#### Selekce vhodných elementů

---

Nejprve potřebujeme získat takový element, která obsahuje celou tabulku se všemi hráči. Projdeme tedy dostupné metody a zkusíme nějakou vybrat:

In [5]:
help(BeautifulSoup.find)

NameError: name 'BeautifulSoup' is not defined

In [4]:
help(BeautifulSoup.find_all)

NameError: name 'BeautifulSoup' is not defined

<br>

#### Selekce tabulky s atributem `tab_top`

---

In [3]:
table_tag_top = soup.find("table", {"class": "tab_top"})

NameError: name 'soup' is not defined

In [ ]:
print(table_tag_top.prettify())

In [ ]:
dir(table_tag_top)

<br>

Jakmile máme celou tabulku, snažíme se rozptyl zmenšit a najít pouze tagy s jednotlivými hráči. Vidíme, že každý hráč je na řádku. Dále, že každý řádek je schovaný za tagem `tr`.

<br>

#### Selekce elementů `tr`, řádků

---

In [ ]:
vsechny_tr = table_tag_top.find_all("tr")
len(vsechny_tr)

In [ ]:
dir(vsechny_tr)

In [ ]:
print(vsechny_tr[1])

In [ ]:
type(vsechny_tr)

In [ ]:
type(vsechny_tr[1])

In [ ]:
prvni_tr = table_tag_top.find("tr")
print(prvni_tr)

In [ ]:
print(prvni_tr.text) # vrátí text ze všech potomků daného elementu


<br>

Nyní, když umíme rozdělit html na jednotlivé řádky (s jednotlivými hráči), potřebujeme konečně izolovat jednotlivé buňky, které obsahují konkrétní údaje.

<br>

#### Selekce jednotlivých buněk `td`

---

In [ ]:
td_na_radku = vsechny_tr[1].find_all("td")
print(td_na_radku[4].get_text())  # získá text z konkrétního elementu


In [ ]:
print(td_na_radku)

In [ ]:
type(td_na_radku)

In [ ]:
print(td_na_radku[2].get_text())

In [ ]:
print(td_na_radku[2].text)

In [ ]:
def vyber_atributy_z_radku(tr_tag: "bs4.element.ResultSet"):

    return {
        "poradi": tr_tag[0].get_text(),
        "jmeno": tr_tag[2].get_text(),
        "vitezstvi": tr_tag[5].get_text(),
        "celkem_her": tr_tag[6].get_text()
    }

In [ ]:
# klasický for loop
vysledky = []

for tr in vsechny_tr[1:]: # hlavičku nechceme
    td_na_radku = tr.find_all("td")
    data_hrace = vyber_atributy_z_radku(td_na_radku)
    vysledky.append(data_hrace)

print(vysledky)

<br>

Podobně jako u řetězit built-in funkce `print(type(...))` můžeme řetězit uživatelské funkce.

In [ ]:
# zkrácený zápis
data_o_hracich = list()

for tr in vsechny_tr[1:]:
    data_o_hracich.append(vyber_atributy_z_radku(tr.find_all("td")))

print(data_o_hracich)

<br>

#### Ukládání údajů do souboru

---

Úplně nakonec chceme námi posbírané údaje uložit ve formátu `csv` do příslušného souboru:

In [ ]:
import csv
import traceback


def zapis_data(data: list, jmeno_souboru: str) -> str:
    """
    Zkus zapsat udaje z par. 'data' do souboru formatu .csv.
    """
    with open(jmeno_souboru, mode="w", encoding="utf-8", newline="") as csv_soubor:
        sloupce = data[0].keys()

        zapis = csv.DictWriter(csv_soubor, fieldnames=sloupce)
        zapis.writeheader()
        zapis.writerows(data)


### S velkou mocí příchází velká zodpovědnost

---

1. Než se pustíme do scrapingu, zkontolujeme `robots.txt` (allow/disallow), např. https://www.idnes.cz/robots.txt
2. Zkontrolujeme, jestli služba nenabízí API (není třeba scrapovat, ale využít endpoint)
3. Nezatěžujeme server velkým množstvím dotazů v krátkém časovém horizontu (pád->blokování)
4. Nepoužívám cizí data ke komerčním účelům (_pravidla&podmínky_)
5. requests, bs4, urllib.requests, htmlparser, requests-html, scrapy, selenium, aj.

<br>

### Závěrem

---

<br>

#### Kam dál?
1. Dejte tomu čas, nechte to sednout,
2. projít ještě jednou jednotlivé lekce a úlohy,
3. komunikace v rámci Slacku,
4. aktualizace [portálu learn.engeto.com](https://learn.engeto.com) úpravy, nové úkoly, teorie.

<br>

#### Jak se zdokonalovat?
1. Čtení + psaní + diskuze (Slack zůstává aktivní),
2. svatá trojice (*algorithm, data structures, libraries*),
3. **nekopírujte projekty ostatních!** Vymyslete, co by se mohlo vám/vašim známým hodit.

<br>

#### Je libo přitvrdit?
1. Git, přík.řádek, IDE, textové editory, virt. prostředí, SQL, pip3, OOP, další knihovny,..
2. pokračovat v další teorii jenom pokud si budete v základních a mírně pokročilých pojmech jistější,
3. Kurz **datové analýzy**,
4. **OOP Python**.

<br>

#### Komunita?
1. [pyvo](https://pyvo.cz/) - organizování setkání po celé Čoro
2. [junior.guru](https://junior.guru/jobs/) (autor: Honza Javorek)
3. [příručka o hledání první práce v IT](https://junior.guru/candidate-handbook/) (autor: Honza Javorek)

---